In [1]:
from utils import *

import os

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [25]:
# Misc
LOG_DIR = './log_pt/'

# Training Parameters
learning_rate = 0.001
num_epochs = 50
batch_size = 256
display_step = 10

# Network Parameters
num_classes = 6
keep_rate = 1

repeat_size = 82

In [26]:
raw = pd.read_csv('./vultures.csv', header=None)

In [27]:
mean = np.array([0. for i in  range(3)])
X = []
Y = []

for i in range(len(raw)):
    
    row = np.array(raw.loc[i][~raw.loc[i].isnull()])
    data = row[0:-1].astype(float)
    label = row[-1]
    
    x_axis = data[0::3]
    y_axis = data[1::3]
    z_axis = data[2::3]
    
    x_axis = repeat_crop_data (x_axis, repeat_size)
    y_axis = repeat_crop_data (y_axis, repeat_size)
    z_axis = repeat_crop_data (z_axis, repeat_size)
    
    mean[0] += x_axis.mean()
    mean[1] += y_axis.mean()
    mean[2] += z_axis.mean()
    
    X.append(np.stack((x_axis, y_axis, z_axis)))
    Y.append(label)

mean = mean/len(raw)
    
X = np.array(X).transpose((0, 2, 1))
Y = np.array(Y)
label_names = np.unique(Y)

le = preprocessing.LabelEncoder()
le.fit(np.unique(Y))
Y = le.transform(Y)

In [28]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

In [29]:
class acc_dataset (torch.utils.data.Dataset):
    
    def __init__ (self, X, Y, mean):
        super().__init__()
        self.X = X
        self.Y = Y.astype(np.float)
        self.mean = mean
        
    def __len__ (self):
        return len(self.X)
    
    def __getitem__ (self, idx):
        
        x = self.X[idx]
        y = self.Y[idx]
        
        x = (x - self.mean).transpose(1, 0)
            
        return torch.tensor(x), torch.tensor(y).type(torch.LongTensor)

In [30]:
train_set = acc_dataset(train_x, train_y, mean)
test_set = acc_dataset(test_x, test_y, mean)

In [59]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [32]:
class MyNet (nn.Module):
    
    def __init__ (self, num_classes, input_dim):
        super().__init__()
        
        self.input_dim = input_dim
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc3 = nn.Linear(self.input_dim*64, 1024)
        self.fc4 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        #self.pretrained_resnet.avgpool(x)
        x = x.view(-1, self.input_dim*64)
        
        x = F.relu(self.fc3(x))
        x = self.fc4(x)

        return x

In [33]:
net = MyNet(num_classes=num_classes, input_dim=repeat_size).double()

In [34]:
# Loss function
loss_function = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 30 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [35]:
step = 0

net.train()

for epoch in range(num_epochs):
    
    for data in train_loader:
        
        x, y = data
        
        optimizer.zero_grad()
        
        logits = net(x)
        
        loss = loss_function(logits, y)
        
        loss.backward()
        
        optimizer.step()
        
        step += 1
        
        _, preds = torch.max(logits, 1)
        
        correct = (preds == y).sum().item()
        
        accuracy = correct/len(x)
        
        if step % display_step == 0:
            print('Epoch: %d, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (epoch, step, loss, accuracy))
            
print('Final, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (step, loss, accuracy))

Epoch: 4, Step: 10, Minibatch loss: 0.867, Minibatch accuracy: 0.743
Epoch: 9, Step: 20, Minibatch loss: 0.594, Minibatch accuracy: 0.831
Epoch: 14, Step: 30, Minibatch loss: 0.361, Minibatch accuracy: 0.869
Epoch: 19, Step: 40, Minibatch loss: 0.217, Minibatch accuracy: 0.945
Epoch: 24, Step: 50, Minibatch loss: 0.130, Minibatch accuracy: 0.962
Epoch: 29, Step: 60, Minibatch loss: 0.164, Minibatch accuracy: 0.945
Epoch: 34, Step: 70, Minibatch loss: 0.081, Minibatch accuracy: 0.973
Epoch: 39, Step: 80, Minibatch loss: 0.100, Minibatch accuracy: 0.967
Epoch: 44, Step: 90, Minibatch loss: 0.064, Minibatch accuracy: 0.984
Epoch: 49, Step: 100, Minibatch loss: 0.034, Minibatch accuracy: 0.989
Final, Step: 100, Minibatch loss: 0.034, Minibatch accuracy: 0.989


In [37]:
torch.save(net.state_dict(), os.path.join(LOG_DIR, 'model_%s_checkpoint.pth.tar' % 'vulter_plain_ep_50_step_10'))

In [36]:
net.eval()

test_correct = 0

with torch.no_grad():
    
    for data in test_loader:

        test_x, test_y = data

        test_logits = net(test_x)
        _, test_preds = torch.max(test_logits, 1)

        test_correct_batch = (test_preds == test_y).sum().item()

        test_correct += test_correct_batch

print('Test accuracy: %.3f' % (test_correct / len(test_set)))

Test accuracy: 0.878


In [60]:
# Training Parameters
learning_rate = 0.01
num_epochs = 50

In [61]:
checkpoint = torch.load('./log_pt/model_storks_plain_ep_60_step_10_checkpoint.pth.tar')

In [62]:
ft_net = MyNet(num_classes=num_classes, input_dim=repeat_size).double()

In [63]:
ft_net.load_state_dict(checkpoint)

In [64]:
# Optimizer
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [65]:
step = 0

ft_net.train()

for epoch in range(num_epochs):
    
    for data in train_loader:
        
        x, y = data
        
        optimizer.zero_grad()
        
        logits = ft_net(x)
        
        loss = loss_function(logits, y)
        
        loss.backward()
        
        optimizer.step()
        
        step += 1
        
        _, preds = torch.max(logits, 1)
        
        correct = (preds == y).sum().item()
        
        accuracy = correct/len(x)
        
        if step % display_step == 0:
            print('Epoch: %d, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (epoch, step, loss, accuracy))
            
print('Final, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (step, loss, accuracy))

Epoch: 4, Step: 10, Minibatch loss: 37.308, Minibatch accuracy: 0.038
Epoch: 9, Step: 20, Minibatch loss: 37.112, Minibatch accuracy: 0.044


Process Process-510:
Process Process-509:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ranranking/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/ranranking/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/ranranking/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ranranking/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ranranking/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/Users/ranranking/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/Users/ranranking/miniconda3/lib/python3.6/multiprocessing/q

KeyboardInterrupt: 

In [ ]:
ft_net.eval()

test_correct = 0

with torch.no_grad():
    
    for data in test_loader:

        test_x, test_y = data

        test_logits = ft_net(test_x)
        _, test_preds = torch.max(test_logits, 1)

        test_correct_batch = (test_preds == test_y).sum().item()

        test_correct += test_correct_batch

print('Test accuracy: %.3f' % (test_correct / len(test_set)))